In [1]:
# Import Dependencies
import pymongo
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
from splinter import Browser

In [2]:
# Find path to our chrome browser

# !which chromedriver

In [39]:
# Set path to browser & start splinter

# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome')

In [40]:
# Open webpage

url = 'http://quotes.toscrape.com/'
browser.visit(url)

In [32]:
# Create BS object

html = browser.html
soup_obj = soup(html, 'html.parser')

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [ ]:
db = client.<database name>.db
collection = db.<collection name>

In [33]:
# Look at html
results = soup_obj.find_all("body")
results

[<body>
 <div class="container">
 <div class="row header-box">
 <div class="col-md-8">
 <h1>
 <a href="/" style="text-decoration: none">Quotes to Scrape</a>
 </h1>
 </div>
 <div class="col-md-4">
 <p>
 <a href="/login">Login</a>
 </p>
 </div>
 </div>
 <div class="row">
 <div class="col-md-8">
 <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
 <span>by <small class="author" itemprop="author">Albert Einstein</small>
 <a href="/author/Albert-Einstein">(about)</a>
 </span>
 <div class="tags">
             Tags:
             <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
 <a class="tag" href="/tag/change/page/1/">change</a>
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>
 <a class="tag" hr

In [34]:
# The loops works but theres one problem with page 3 where it doesn't
# retrieve the author information (descrp, location, and date) so I had
# to write a try function to work around that and add something in its
# place. Even though the loop works the tags are still how they were 
# before. One thing we'll need to fix is how to stop the loop when we get
# to the end of the pages. The only way I could do it is if I tell it to 
# stop at 10.

# final_data = []

text_list = []
tags_list = []
author_list = []
author_date_list = []
author_location_list = []
final_decrp_list = []
count = 0
while True:
    count += 1
    for stuff in results:
        
        html = browser.html
        soup_obj = soup(html, 'html.parser')
        
        quote_block = soup_obj.findAll("div", class_="quote")
#         for block in quote_block:
#             block_list.append(block.text) 
#             block_list = block.text
    
#         Finds quotes
        quote_text = soup_obj.findAll("span", class_ = "text")
        for text in quote_text:
            text_list.append(text.text) 
#             text_list = text.text
            
#         Finds tags
        quote_tags = soup_obj.findAll("a", class_ = "tag")
        for tags in quote_tags:
            tags_list.append(tags.text) 
#             tags_list = tags.text
            
#         Finds author
        quote_author = soup_obj.findAll("small", class_="author")
        for author in quote_author:
            author_list.append(author.text)
#             author_list = author.text
        

        
    
        
# # # # # # This is an href list but I had to get rid of all the tags I knew wouldn't be needed.
        links_list = []
        links = soup_obj.find_all('a')
        for link in links:
            links_list.append(link['href'])

        links_list.pop(0) # Pesky forward slash that wasn't needed
        links_list = [x for x in links_list if not 'tag' in x] # Got rid of tag links
        links_list = [x for x in links_list if not 'page' in x] # Got rid of page links
        links_list = [x for x in links_list if not 'login' in x] # Got rid of login link
        links_list = [x for x in links_list if not 'https' in x] # Got rid of https links


        for click in links_list:
            try:
                browser.click_link_by_href(click)
            except Exception:
                pass 
            
            html = browser.html
            soup_obj = soup(html, 'html.parser')    

            author_date = soup_obj.find('span', class_="author-born-date")
            author_location = soup_obj.find('span', class_="author-born-location")
            author_descrp = soup_obj.find('div', class_="author-description")
            try:
                author_date_list.append(author_date.text)
                author_location_list.append(author_location.text)
                final_decrp_list.append(author_descrp.text)
                browser.back()
#             except Exception:
#                 pass
            except:
                author_date_list.append('*****Could Not Retrieve*****')
                author_location_list.append('*****Could Not Retrieve*****')
                final_decrp_list.append('*****Could Not Retrieve*****')  
            
        

# # # # # # Next Page 
        html = browser.html
        soup_obj = soup(html, 'html.parser')
        page_link = []
        all_links = soup_obj.find_all('a')
        for p_link in all_links:
            page_link.append(p_link['href'])
        page_link = [x for x in page_link if not 'tag' in x]
        page_link = [x for x in page_link if 'page' in x]
        try:
            browser.click_link_by_href(page_link[1])
        except:
            browser.click_link_by_href(page_link[0])

    if count == 10:
        break
    print(count)

1
2
3
4
5
6
7
8
9


In [36]:
# # # # # # Made sure lists were being filled

# text_list
# tags_list
# author_list
author_date_list
# author_location_list
# final_decrp_list
# final_data

# author_list
# This was the last thing I did. The rest is unchanged from
# when we did it in class.

['March 14, 1879',
 'July 31, 1965',
 'March 14, 1879',
 'December 16, 1775',
 'June 01, 1926',
 'March 14, 1879',
 'November 22, 1869',
 'February 11, 1847',
 'October 11, 1884',
 'August 14, 1945',
 'June 01, 1926',
 'July 31, 1965',
 'March 14, 1879',
 'February 06, 1945',
 'March 02, 1904',
 'March 11, 1952',
 'September 30, 1928',
 'October 15, 1844',
 'November 30, 1835',
 'April 24, 1899',
 'July 12, 1904',
 'May 25, 1803',
 'August 26, 1910',
 'August 07, 1942',
 'September 24, 1936',
 'March 02, 1904',
 'March 14, 1879',
 'July 31, 1965',
 'March 14, 1879',
 'February 06, 1945',
 'March 02, 1904',
 'July 31, 1965',
 'February 06, 1945',
 'August 26, 1910',
 'July 31, 1965',
 'November 26, 1922',
 'January 12, 1948',
 'March 14, 1879',
 'August 24, 1899',
 'November 22, 1819',
 'September 20, 1948',
 'November 29, 1898',
 'June 01, 1926',
 'June 01, 1926',
 'March 14, 1879',
 'June 01, 1926',
 'June 01, 1926',
 'January 15, 1929',
 'July 31, 1965',
 'August 02, 1924',
 'Decembe

In [ ]:
all_data = {
    'author_name':author_list,
    'text':text_list,
    'tag':tags_list,
    'born':author_date_list,
    'location':author_location_list,
    'description':final_decrp_list
}


In [21]:
new_final_list = []
for x in final_decrp_list:
    new_list = x.split('\n')[1].strip()
    new_final_list.append(new_list)

IndexError: list index out of range